In [ ]:
! pip install SQLAlchemy==2.0.29

INSTAMOS CON PIP

In [ ]:
! pip install SQLAlchemy==2.0.29

In [ ]:
! pip install mysql-connector-python==8.3.0


In [ ]:
! pip install mysqlclient==2.2.4

In [ ]:
! pip install pandas

IMPORTAMOS LAS LIBERIAS 

In [ ]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import text
from  utils import connection as con
from etl import extraccion as ex
from etl import carga
from datetime import date

In [ ]:
conn_cyber_db = con.connect_cyber_db()
conn_dw_cyber = con.connect_dw_cyber()

HACEMOS LA EXTRACCION DE LAS TABLAS OLTP

In [ ]:
df_caja = ex.extract_table("caja", conn_cyber_db)
df_operadores= ex.extract_table("operadores", conn_cyber_db)
df_categoriascaja = ex.extract_table("categoriascaja", conn_cyber_db)
df_movimientoscaja = ex.extract_table("movimientoscaja", conn_cyber_db)


INSERTAMOS A LA DIMENSION CAJA

In [ ]:
df_caja = df_caja.rename(columns={'fechacaja': 'fecha_caja','fechacierre': 'fecha_cierre' })
df_caja.head()

In [ ]:
carga.load_dw_cyber('dim_caja',conn_dw_cyber, df_caja)

INSERTAMOS A LA DIMENSION OPERADOR

In [ ]:
df_operadores = df_operadores.rename(columns={'Permisos': 'permiso'})
df_operadores.head()

In [ ]:
carga.load_dw_cyber('dim_operador',conn_dw_cyber, df_operadores)

INSERTAMOS DIMENSION CATEGORIA CAJA

In [ ]:
df_categoriascaja.head()

In [ ]:
query = """
SELECT Codigo, id, fijo, tipo, Nombre, NOW() fecha_carga FROM cyber.categoriascaja
"""

In [ ]:
categoriascaja_data=pd.read_sql(query,con=conn_cyber_db)
categoriascaja_data.head()

In [ ]:
carga.load_dw_cyber('dim_categoriacaja',conn_dw_cyber, categoriascaja_data)

INSERTAMOS LA DIM TIEMPO

In [ ]:
query_time ="""
SELECT
    DISTINCT DATE_FORMAT(A.FECHA, '%%Y%%m%%d') AS FECHA
    ,DATE(A.FECHA) AS DATE
    ,YEAR(A.FECHA) AS year
    ,QUARTER(A.FECHA) AS quarter
    ,MONTH(A.FECHA) AS month
    ,DAY(A.FECHA) AS day
    ,DAYOFWEEK(A.FECHA) AS day_of_week
    ,DAYOFYEAR(A.FECHA) AS day_of_year
    ,WEEK(A.FECHA) AS week_of_year
    ,CASE WHEN DAYOFWEEK(A.FECHA) IN (1,7) THEN TRUE ELSE FALSE END AS is_weekend
FROM CYBER.movimientoscaja  A 
"""

In [ ]:
tiempo_data=pd.read_sql(query_time,con=conn_cyber_db)
tiempo_data.head()

In [ ]:
carga.load_dw_cyber('dim_tiempo',conn_dw_cyber, tiempo_data)

INSERTAMOS O CARAGAMOS LA TABLA DE HECHO

In [ ]:
query ="""
 SELECT DATE_FORMAT(mov.Fecha, '%%Y%%m%%d') AS dim_tiempo_fecha
      , COALESCE(ope.idw_operador,0) as dim_operador_idw_operador
      , COALESCE(catc.idw_catcaja,0) as dim_categoriacaja_idw_catcaja
      , COALESCE(caj.idw_caja,0) as dim_caja_idw_caja
      , mov.Movimiento  as importe_movimiento
      , case when mov.Movimiento >=0 then  mov.Movimiento else 0 end as importe_positivo
      , case when mov.Movimiento <0 then  mov.Movimiento else 0 end as importe_negativo
      , mov.Cantidad
      , mov.Concepto
      , 1 as cantidad_reg
      ,now() fecha_carga      
  FROM CYBER.movimientoscaja mov
  left join dwh_lancenter.dim_operador ope on (mov.Operador=ope.nombre)
  left join dwh_lancenter.dim_categoriacaja catc on (mov.codcategoria=catc.codigo)
  left join dwh_lancenter.dim_caja caj on (mov.ncaja=caj.ncaja)
"""

In [ ]:
fact_mov_data=pd.read_sql(query,con=conn_dw_cyber)
fact_mov_data.head()

In [ ]:
fact_mov_data.to_sql('fact_movimiento_caja',conn_dw_cyber, if_exists='append', index=False)

RESOLVIENDO 5 CONSULTAS CON RESPECTO A MODELO OLAP



In [85]:
#-- top operadores que generan mas importe al Lan center año actual
query1= """
select dt.year,  doper.nombre, sum(fm.importe_positivo) importe
from dwh_lancenter.fact_movimiento_caja fm
left join dwh_lancenter.dim_operador doper on (fm.dim_operador_idw_operador=doper.idw_operador)
left join dwh_lancenter.dim_tiempo dt on (dt.fecha=fm.dim_tiempo_fecha)
where dt.year='2024'
group by dt.year,  doper.nombre
order by 3 desc
"""

In [86]:
query1_df=pd.read_sql(query1,con=conn_dw_cyber)
query1_df

,year,nombre,importe
0,2024,Jhessy,11468.5
1,2024,Dennis,8142.6
2,2024,Juan,3468.0
3,2024,Daniel21,2878.5
4,2024,Ariel,547.5
5,2024,Mary,247.4


In [87]:
#-- en que categoria caja se generean mas importes negativos 

query2= """select dt.year, dc.nombre as categoria_caja, sum(fm.importe_negativo) as importe_negativo
from dwh_lancenter.fact_movimiento_caja fm
left join dwh_lancenter.dim_categoriacaja dc  on (dc.idw_catcaja=fm.dim_categoriacaja_idw_catcaja)
left join dwh_lancenter.dim_tiempo dt on (dt.fecha=fm.dim_tiempo_fecha)
where dt.year>='2023'
and fm.importe_negativo<0
group by dt.year, dc.nombre
order by 3 asc
"""

In [88]:
query2_df=pd.read_sql(query2,con=conn_dw_cyber)
query2_df

,year,categoria_caja,importe_negativo
0,2023,Retiro de caja,-96082.50
1,2024,Retiro de caja,-25396.00
2,2024,Cierre Caja,-945.00
3,2023,Cierre Caja,-394.50
4,2024,Descuadre,-298.50
5,2023,Descuentos,-51.03
6,2023,Caja entregada,-28.50
7,2023,Descuadre,-18.50


In [89]:
#-- que mes se genero mas imgreso (importe total ) en los 2 ultimos años
query3="""
select dt.year, dt.month,  sum(fm.importe_positivo)
from dwh_lancenter.fact_movimiento_caja fm
left join dwh_lancenter.dim_tiempo dt on (dt.fecha=fm.dim_tiempo_fecha)
where dt.year>='2023'
group by  dt.year, dt.month
order by 3 desc
"""

In [90]:
query3_df=pd.read_sql(query3,con=conn_dw_cyber)
query3_df

,year,month,sum(fm.importe_positivo)
0,2023,11,10104.00
1,2023,10,9254.00
2,2024,3,9203.00
3,2023,8,9166.50
4,2023,5,9048.00
5,2023,3,8881.03
6,2023,4,8544.50
7,2023,9,8072.00
8,2023,6,7736.50
9,2024,2,7330.00


In [93]:
# generar reporte de importe por caterigoria caja (importe positivo) ultimo  año
query4="""
select dt.year,   dc.nombre as categoria_caja, sum(fm.importe_positivo)
from dwh_lancenter.fact_movimiento_caja fm
left join dwh_lancenter.dim_categoriacaja dc  on (dc.idw_catcaja=fm.dim_categoriacaja_idw_catcaja)
left join dwh_lancenter.dim_tiempo dt on (dt.fecha=fm.dim_tiempo_fecha)
where dt.year>='2024'
and fm.importe_positivo>0
group by dt.year,   dc.nombre
order by 3 desc
"""

In [92]:
query4_df=pd.read_sql(query4,con=conn_dw_cyber)
query4_df

,year,categoria_caja,sum(fm.importe_positivo)
0,2024,Uso Libre,6440.5
1,2024,Productos,6388.5
2,2024,Cargas de abonos,5633.0
3,2024,Impresiones color,4059.0
4,2024,Carga Prepago,2555.5
5,2024,Impresiones BN,1201.5
6,2024,Servicios,400.5
7,2024,Ajuste por redondeo,64.5
8,2024,Escaneos,9.5


In [94]:
#cuando veces se logearon los operador por mes en el ultimo año
query5="""
select dt.year, dt.month,  doper.nombre, count(distinct dim_categoriacaja_idw_catcaja) cant_logueo
from dwh_lancenter.fact_movimiento_caja fm
left join dwh_lancenter.dim_operador doper on (fm.dim_operador_idw_operador=doper.idw_operador)
left join dwh_lancenter.dim_tiempo dt on (dt.fecha=fm.dim_tiempo_fecha)
where dt.year='2024'
group by dt.year,dt.month,   doper.nombre
order by 2 desc
"""

In [95]:
query5_df=pd.read_sql(query5,con=conn_dw_cyber)
query5_df

,year,month,nombre,cant_logueo
0,2024,4,Ariel,1
1,2024,4,Daniel21,13
2,2024,4,Dennis,12
3,2024,4,Jhessy,12
4,2024,4,Juan,12
5,2024,3,Ariel,12
6,2024,3,Daniel21,14
7,2024,3,Dennis,13
8,2024,3,Jhessy,13
9,2024,3,Juan,12
